In [233]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


from scipy.stats import skew
from scipy.special import boxcox1p

from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score

import re

from IPython.display import display
pd.options.display.max_columns = 500

## Import data set

In [234]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## Outliers

We need to add justification

In [235]:
# remove outliers
train = train[~((train['GrLivArea'] > 4000) & (train['SalePrice'] < 300000))]

## Remove SalePrice and Id

In [236]:
response_variable = np.log1p(train.SalePrice)
id_ = test.Id
train = train.loc[:,'MSSubClass':'SaleCondition']
test = test.loc[:,'MSSubClass':'SaleCondition']

## Multicoliearity 

We need to add justification

In [237]:
# drop some features to avoid multicollinearity
train.drop(['1stFlrSF', 'GarageArea', 'TotRmsAbvGrd'], axis=1, inplace=True)
test.drop(['1stFlrSF', 'GarageArea', 'TotRmsAbvGrd'], axis=1, inplace=True)

## Skewdness

In [238]:
numeric_feats = train.dtypes[train.dtypes != "object"].index

In [239]:
skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.65]
skewed_feats = skewed_feats.index

train[skewed_feats] = boxcox1p(train[skewed_feats], 0.15)
test[skewed_feats] = boxcox1p(test[skewed_feats], 0.15)

## Dummification

In [240]:
train = pd.get_dummies(train)
train = train.fillna(train.mean())

In [241]:
test = test.fillna(test.mean())

In [242]:
test = pd.get_dummies(test)

In [243]:
train_cols = set(train.columns)
test_cols = set(test.columns)

test_drop =  list(test_cols - train_cols)
test_unkonwn_dummies = list(train_cols - test_cols)

In [244]:
test_dummies2 = test.reindex(columns=test_unkonwn_dummies, fill_value=0)

In [245]:
test = pd.concat([test, test_dummies2], axis=1)

In [246]:
test = test[train.columns]

In [247]:
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

## Train Test Split

In [248]:
#split
X_train = train
X_test = test
y = response_variable

## Modeling

In [249]:
### models selection
lasso = Lasso(alpha=0.0004)
model = lasso

## Prediciton

In [250]:
model.fit(X_train, y)
preds = np.expm1(model.predict(X_test))
solution = pd.DataFrame({"id":id_, "SalePrice":preds})
solution.to_csv("/home/gavagai/jails/chromium/Downloads/full_features_lasso.csv", index = False)